---
title: "Bilanci imprese - ratios regionali"
author: Paolo Volterra
created: 2025-06-24
updated: 2025-06-27
affiliation: "UO Studi e Governo Iniziative"
subtitle: "Analisi per area geografica basata su dati 2018-2023 Banca d'Italia"
keywords: [bilanci, imprese, Bankit, indici, ratios, leverage, ROE, ROA, PFN, patrimonializzazione, mezzipropri, indebitamento]
jupyter: python3
format:
  html:
    code-fold: true # Nasconde il codice ma lo rende espandibile/collassabile 
    css: D:/Python/Eurostat/style1.css # MCC_ChatGPT.css
    theme: none
    toc: true
    number-sections: true
    echo: false  # nasconde tutto il codice Python
    self-contained: true     # <--- TUTTO incluso nel file HTML
---

<br><br> <!-- due righe di spazio -->

La collana [Economie regionali](https://www.bancaditalia.it/pubblicazioni/economie-regionali/index.html) di banca d'Italia contiene analisi sulle principali articolazioni territoriali (regioni e macroaree) dell'economia italiana.

Una delle tabelle riportate in appendice riporta i dati sugli "Indicatori economici e finanziari delle imprese"

![](./media/immagine001.png)


Il presente studio vuole analizzare l'andamento degli indicatori per regione nel tempo.  
Un focus viene fatto sul credit crunch.

<br><br> <!-- due righe di spazio -->

---

## Dati 

In [ ]:
import pandas as pd, os
os.chdir('D:/')
file_path = "Bankit_ratios_2025H1.tsv"
df = pd.read_csv(file_path, sep="\t")
df.columns = ["REGIONE", "RATIO", "ANNO", "VALORE"]
df["VALORE"] = df["VALORE"].str.replace(",", ".").astype(float)
mezzogiorno = {"ABR", "BAS", "CAL", "CAM", "PUG", "SAR", "SIC", "MOL"}
df["MACROAREA"] = df["REGIONE"].apply(lambda x: "Mezzogiorno" if x in mezzogiorno else "Centro-Nord")
# Calcolo della media per macroarea, ratio e anno
df_grouped = df.groupby(["MACROAREA", "RATIO", "ANNO"])["VALORE"].mean().reset_index()
# Pivot per confronto tra Mezzogiorno e Centro-Nord
df_pivot = df_grouped.pivot(index=["RATIO", "ANNO"], columns="MACROAREA", values="VALORE").reset_index()
df_pivot

<br><br> <!-- due righe di spazio -->

### Indicatori presenti
Il dataset contiene rapporti finanziari chiave, tra cui:

- Debiti bancari / Debiti finanziari: incidenza dei prestiti bancari sul totale dei debiti finanziari.
- Oneri finanziari / Valore della produzione: peso del costo del debito rispetto al fatturato.
- Patrimonio netto / Totale delle passività: misura della solidità patrimoniale.
- Indebitamento finanziario / Attivo: leva finanziaria.
- Autofinanziamento / Investimenti: capacità interna di finanziare la crescita.

Le imprese del Mezzogiorno risultano più dipendenti dal credito bancario, con struttura finanziaria più fragile e costo del debito più elevato.

Ciò segnala un ritardo nell’accesso alla finanza alternativa e nella capacità di autofinanziamento.


### Debiti bancari / Debiti finanziari
Costantemente più alto nel Mezzogiorno: segno che le imprese meridionali fanno maggiormente ricorso al credito bancario tradizionale, con meno accesso a strumenti alternativi (leasing, factoring, minibond).

Il gap è stabile: ~10–15 punti percentuali a favore del Centro-Nord.

### Oneri finanziari / Valore della produzione
Più elevati nel Mezzogiorno, con un divario che si allarga nel 2022–2023.

Indica che il costo del denaro pesa di più per le imprese meridionali, a parità di fatturato: possibile effetto combinato di rating più deboli, minore concorrenza bancaria, meno strumenti di copertura.

### Patrimonio netto / Totale delle passività
Centro-Nord meglio capitalizzato: rapporto stabilmente superiore, anche in anni critici.

Le imprese del Sud mostrano una minore patrimonializzazione, quindi maggiore vulnerabilità in caso di shock o aumento dei tassi.

### Autofinanziamento / Investimenti
Il Centro-Nord finanzia una quota più ampia degli investimenti con risorse proprie.

Il Mezzogiorno mostra maggiore dipendenza da fonti esterne (pubbliche o bancarie), che rende l’investimento più volatile.

### Evoluzione temporale (2018–2023)

- Post-Covid (2020–2021): leggero miglioramento in molti indicatori, probabilmente per effetto di moratorie, liquidità straordinaria, fondo di garanzia.
- 2022–2023: inizia a emergere un deterioramento, con peggioramento degli oneri finanziari e lieve riduzione della patrimonializzazione.

<br><br> <!-- due righe di spazio -->

## Cluster analysis 

In [ ]:
#| label: fig1
#| fig-cap: curva Elbow
#| fig-align: center

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# === Preparazione dati: media 2018–2023 per regione e ratio
df_cluster = df.groupby(["REGIONE", "RATIO"])["VALORE"].mean().unstack()

# === Rimozione regioni con valori NaN
df_cluster_clean = df_cluster.dropna()

# === Standardizzazione
scaler = StandardScaler()
X_scaled_clean = scaler.fit_transform(df_cluster_clean)

# === Metodo del gomito per scelta K ottimale
K_range = range(1, 10)
inertia_clean = []
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
    kmeans.fit(X_scaled_clean)
    inertia_clean.append(kmeans.inertia_)

# === Visualizzazione gomito
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia_clean, marker='o', color='green')
plt.xlabel("Numero di cluster (k)")
plt.ylabel("Inertia")
plt.title("Elbow curve")
plt.grid(True)
plt.ylim(min(inertia_clean) * 0.95, max(inertia_clean) * 1.05)  # zoom verticale
plt.show()

In [ ]:
# === Clustering finale (K=3)
kmeans_final = KMeans(n_clusters=3, random_state=0, n_init=10)

# Copia sicura per evitare ambiguità
df_cluster_clean = df_cluster_clean.copy()
df_cluster_clean["CLUSTER"] = kmeans_final.fit_predict(X_scaled_clean)

# === Valori medi per cluster
# Valori medi per cluster (trasposti)
df_cluster_mean = df_cluster_clean.groupby("CLUSTER").mean().round(2)
df_cluster_mean_t = df_cluster_mean.T.sort_index()

# Uso della libreria tabulate per mostrare una tabella ben formattata in stile "pretty"
from tabulate import tabulate

print(tabulate(df_cluster_mean_t.reset_index(), headers="keys", tablefmt="github", showindex=False))

df_cluster_mean_t.to_html("cluster_table.html", float_format="%.2f")

<br><br> <!-- due righe di spazio -->

La segmentazione delle regioni italiane in 3 cluster basata sui loro indicatori finanziari medi (2018–2023). Ogni cluster rappresenta un profilo omogeneo di comportamento finanziario:

- Cluster 0: regioni con alta dipendenza da debito bancario, leva moderata, e autofinanziamento medio.
- Cluster 1: (da identificare dopo confronto completo) – potenzialmente regioni con maggiore solidità patrimoniale.
- Cluster 2: (da verificare) – potrebbe contenere regioni con performance più efficienti o più fragili, a seconda della direzione dei ratios.

<br><br> <!-- due righe di spazio -->

In [ ]:
# Eseguo il clustering con k=3
kmeans_final = KMeans(n_clusters=3, random_state=0, n_init=10)
df_cluster_clean["CLUSTER"] = kmeans_final.fit_predict(X_scaled_clean)

# Riordino per cluster
df_cluster_clean_sorted = df_cluster_clean.sort_values("CLUSTER")

# Visualizzo i risultati
df_cluster_clean_sorted.reset_index()

<br><br> <!-- due righe di spazio -->

### 🟢 Cluster 0: "Alta dipendenza dal credito bancario" - Caratteristiche:

-Rapporto Debiti bancari / Debiti finanziari molto elevato.
-Costi finanziari / valore della produzione alti, segno di un onere del debito significativo.
-Leva finanziaria (Debiti finanziari / Attivo) moderata-alta.
-Autofinanziamento / Investimenti nella media: buona capacità di generare risorse ma non sufficiente a coprire pienamente gli investimenti.
-Buona generazione di cassa, capacità applicativa di leva	Dipendenza da banche, vulnerabilità all’aumento tassi
- Profilo: imprese con forte esposizione al credito bancario, costi finanziari superiori alla media, leva e autofinanziamento nella media

### 🟡 Cluster 1: "Solidità patrimoniale e rifinanziamento bilanciato" - Caratteristiche:

- Rapporto Patrimonio netto / Totale passività alto, indica buona capitalizzazione.
- Debiti bancari / Debiti totali più contenuti rispetto al Cluster 0, con maggiore presenza di strumenti finanziari diversificati.
- Autofinanziamento / Investimenti elevato: le imprese tendono a finanziare internamente gli investimenti.
- Costi finanziari contenuti, in linea o al di sotto della media nazionale.
- Elevata patrimonializzazione, autonomia finanziaria	Potrebbe limitare leva utile per crescite aggressive
- Profilo: imprese con maggiore capitale proprio, accesso più bilanciato alla finanza, migliore posizione finanziaria netta


### 🔴 Cluster 2: "Fragilità accentuata o piccole imprese esposte" - Caratteristiche:

- Leva finanziaria molto elevata, indicando forte indebitamento.
- Autofinanziamento / Investimenti basso: dipendenza significativa da finanziamenti esterni.
- Rapporto Patrimonio netto / Totale passività più basso, indice di fragilità patrimoniale.
- I costi finanziari possono essere molto alti, per l’effetto combinato di leva elevata e rating debole.
- Possibile presenza di realtà dinamiche in espansione	Elevata vulnerabilità finanziaria, rischi elevati
- Profilo: combinazione di leve elevate, bassa patrimonializzazione, autofinanziamento ridotto – possibili anomalie strutturali o effetti della dimensione media d’impresa

In [ ]:
#| label: fig3
#| fig-cap: radar comparativo dei cluster
#| fig-align: center
# Generazione di un radar unico con tutti i cluster sovrapposti

import numpy as np
labels = df_cluster_mean_t.index.tolist()
num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

# Cluster colors e labels
cluster_colors = ['gold', 'green', 'red']
cluster_labels = ['Cluster 0', 'Cluster 1', 'Cluster 2']

# Radar comparativo senza riempimento interno e con immagine più grande

fig, ax = plt.subplots(figsize=(12,12), subplot_kw=dict(polar=True))

for i in range(3):
    values = df_cluster_mean_t.iloc[:, i].tolist()
    values += values[:1]
    ax.plot(angles, values, color=cluster_colors[i], linewidth=2, label=cluster_labels[i])
    # Niente fill per maggiore pulizia visiva

ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, fontsize=10)
ax.set_yticklabels([])
ax.set_title("Grafico Radar dei Cluster", fontsize=16, pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

plt.tight_layout()
plt.show()

In [ ]:
# Estraggo le regioni per ciascun cluster
cluster_regioni = df_cluster_clean_sorted.reset_index()[["REGIONE", "CLUSTER"]]
cluster_regioni_grouped = cluster_regioni.groupby("CLUSTER")["REGIONE"].apply(list).reset_index(name="REGIONI")

cluster_regioni_grouped

In [ ]:
#| label: fig2
#| fig-cap: clusterizzazione regioni
#| fig-align: center
import geopandas as gpd
import matplotlib.pyplot as plt

# Uso un dataset di regioni italiane semplificato da ISTAT (necessario importarlo)
# Carico un dataset esterno con codici e geometrie regionali
regioni = gpd.read_file("https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson")

# Mappo i codici ISO (es. 'IT-62') in codici del file (es. 'VEN') usando una tabella
codici_regioni = {
    'Abruzzo': 'ABR', 'Basilicata': 'BAS', 'Calabria': 'CAL', 'Campania': 'CAM',
    'Emilia-Romagna': 'EMR', 'Friuli-Venezia Giulia': 'FVG', 'Lazio': 'LAZ', 'Liguria': 'LIG',
    'Lombardia': 'LOM', 'Marche': 'MAR', 'Molise': 'MOL', 'Piemonte': 'PIE', 'Puglia': 'PUG',
    'Sardegna': 'SAR', 'Sicilia': 'SIC', 'Toscana': 'TOS', 'Trentino-Alto Adige/Südtirol': 'Tre',
    'Umbria': 'UMB', 'Valle d\'Aosta/Vallée d\'Aoste': 'VAO', 'Veneto': 'VEN',
    'Provincia Autonoma Bolzano/Bozen': 'Bol'
}

regioni["CODICE"] = regioni["reg_name"].map(codici_regioni)

# Unisco i cluster alle geometrie
df_map = df_cluster_clean_sorted.reset_index()[["REGIONE", "CLUSTER"]]
mappa_cluster = regioni.merge(df_map, left_on="CODICE", right_on="REGIONE", how="left")

from matplotlib.colors import ListedColormap
# Plot
fig, ax = plt.subplots(1, 1, figsize=(10, 12))
cmap3 = ListedColormap(["#FFD700", "#228B22", "#B22222"])  # giallo, verde, rosso
mappa_cluster.plot(
    column="CLUSTER",
    cmap=cmap3,
    linewidth=0.8,
    edgecolor="gray",
    legend=False,
    ax=ax
)
ax.set_title("Cluster finanziari delle regioni italiane (2018–2023)", fontsize=14)
ax.axis("off")
plt.show()

## Analisi temporale

Rifacciamo il clustering considerando tutte le annualità (2018–2023) per ogni ratio, quindi:
- Ogni regione è descritta da 103 variabili (ratio_anno), invece della sola media.
- Il clustering cattura anche la dinamica temporale e non solo il valore medio statico.
- Le regioni sono classificate in 3 cluster più coerenti con l’evoluzione nel tempo degli indicatori.

In [ ]:
import pandas as pd
file_path = "D:/Bankit_ratios_2025H1.tsv"
df = pd.read_csv(file_path, sep="\t")
df.columns = ["REGIONE", "RATIO", "ANNO", "VALORE"]
df["VALORE"] = df["VALORE"].str.replace(",", ".").astype(float)
# Pivot table: una colonna per ciascun ratio_anno
df["RATIO_ANNO"] = df["RATIO"] + "_" + df["ANNO"].astype(str)
df_wide = df.pivot(index="REGIONE", columns="RATIO_ANNO", values="VALORE")

# Rimuovo righe con NaN e standardizzo
df_wide_clean = df_wide.dropna().copy()
scaler = StandardScaler()
X_scaled_all_years = scaler.fit_transform(df_wide_clean)

# Clustering
kmeans_all_years = KMeans(n_clusters=3, random_state=0, n_init=10)
df_wide_clean["CLUSTER"] = kmeans_all_years.fit_predict(X_scaled_all_years)

# Salvo la tabella finale
df_cluster_all_years = df_wide_clean.copy()

# Recupero il DataFrame originale
df_clusters_by_year = df.copy()

# Aggiungo la colonna cluster da df_cluster_all_years
cluster_map = df_cluster_all_years["CLUSTER"]
df_clusters_by_year["CLUSTER"] = df_clusters_by_year["REGIONE"].map(cluster_map)

# Tabella finale: regioni in riga, anni in colonna, valore = cluster
df_cluster_matrix = df_clusters_by_year[["REGIONE", "ANNO", "CLUSTER"]].drop_duplicates()
df_cluster_matrix_wide = df_cluster_matrix.pivot(index="REGIONE", columns="ANNO", values="CLUSTER").astype("Int64")

df_cluster_matrix_wide

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# === Caricamento e pulizia dati ===
df = pd.read_csv("D:/Bankit_ratios_2025H1.tsv", sep="\t")
df.columns = ["REGIONE", "RATIO", "ANNO", "VALORE"]
df["ANNO"] = df["ANNO"].str.replace("Y", "")
df["VALORE"] = df["VALORE"].str.replace(",", ".").astype(float)
# === Filtro dati 2018 ===
df_2018 = df[df["ANNO"] == "2018"]

# === Pivot Regione x RATIO ===
df_2018_pivot = df_2018.pivot(index="REGIONE", columns="RATIO", values="VALORE")

# === Imputazione media colonna per valori mancanti ===
df_2018_pivot_filled = df_2018_pivot.fillna(df_2018_pivot.mean(numeric_only=True))

# === Standardizzazione e clustering (K=3) ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_2018_pivot_filled)

kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)
df_2018_pivot_filled["CLUSTER"] = kmeans.fit_predict(X_scaled)

# === Calcolo delle medie per cluster ===
df_cluster_mean_2018 = df_2018_pivot_filled.groupby("CLUSTER").mean().T

# === Definizione grafico radar ===
cluster_colors = ['gold', 'green', 'red']
cluster_labels = ['Cluster 0', 'Cluster 1', 'Cluster 2']
labels = df_cluster_mean_2018.index.tolist()
num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

# === Creazione radar chart ===
fig, ax = plt.subplots(figsize=(14, 14), subplot_kw=dict(polar=True))

for i in range(3):
    values = df_cluster_mean_2018.iloc[:, i].tolist()
    values += values[:1]
    ax.plot(angles, values, color=cluster_colors[i], linewidth=2, label=cluster_labels[i])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, fontsize=9)
ax.set_yticklabels([])
ax.set_title("Grafico Radar dei Cluster - Anno 2018 (dati completati)", fontsize=16, pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

plt.tight_layout()
plt.show()